In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as mlt
import seaborn as sp
from torch.autograd import Variable
from torch import autograd
from datetime import datetime
import matplotlib.pyplot as plt
import argparse
from datetime import timedelta

In [1]:
def save_imputed_data(real_data, impute_data, path, columns, scaler=None):
    real_data_temp = real_data
    impute_data_temp = impute_data

    imputed_dataframe = pd.DataFrame(impute_data_temp.detach().cpu().numpy(), columns=columns)
    real_dataframe = pd.DataFrame(real_data_temp.detach().cpu().numpy(), columns=columns)
    #imputed_dataframe[columns] = real_dataframe[columns]
    if scaler is not None:
        imputed_dataframe = scaler.inverse_transform(imputed_dataframe)
        imputed_dataframe = np.array(imputed_dataframe)
        imputed_dataframe = pd.DataFrame(imputed_dataframe, columns=columns)

    imputed_dataframe['Energy'] = imputed_dataframe['Energy'].apply(lambda x: 0 if x < 0 else x)
    imputed_dataframe['Start'] = imputed_dataframe.apply(lambda row: datetime(int(round(row['Year'])), int(round(row['Month'])), int(round(row['Day of month'])), int(round(row['Hour']))), axis=1)    
    imputed_dataframe.set_index('Start', inplace=True)
    imputed_dataframe = get_data_decomposition(imputed_dataframe)
    imputed_dataframe.reset_index(drop = True, inplace=True)
    # imputed_dataframe['Sum'] = imputed_dataframe.groupby(pd.to_datetime(imputed_dataframe['Start']).dt.date)['Energy'].cumsum()
    # imputed_dataframe.drop(columns=['Start'], inplace=True)
    imputed_dataframe.to_csv(path, index=False) 
